https://gamedevacademy.org/perceptrons-the-first-neural-networks/
https://d2l.ai/
https://maviccprp.github.io/a-neural-network-from-scratch-in-just-a-few-lines-of-python-code/
https://faculty.cc.gatech.edu/~isbell/tutorials/rbf-intro.pdf
https://hal.science/hal-02138762/document

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pre-configuração

In [ ]:
import numpy as np
from numpy import ndarray
import pandas as pd
from pandas import Series,DataFrame
from pandas.core.generic import NDFrame
from matplotlib import pyplot as plt
path = '/content/drive/MyDrive/AV2/magic04.csv'

# Descrição dos rotulos

1.  fLength:  continuous  # major axis of ellipse [mm]
2.  fWidth:   continuous  # minor axis of ellipse [mm] 
3.  fSize:    continuous  # 10-log of sum of content of all pixels [in #phot]
4.  fConc:    continuous  # ratio of sum of two highest pixels over fSize  [ratio]
5.  fConc1:   continuous  # ratio of highest pixel over fSize  [ratio]
6.  fAsym:    continuous  # distance from highest pixel to center, projected onto major axis [mm]
7.  fM3Long:  continuous  # 3rd root of third moment along major axis  [mm] 
8.  fM3Trans: continuous  # 3rd root of third moment along minor axis  [mm]
9.  fAlpha:   continuous  # angle of major axis with vector to origin [deg]
10.  fDist:    continuous  # distance from origin to center of ellipse [mm]
11.  class:    g,h         # gamma (signal), hadron (background)

# Leitura e tratamento dos dados

In [ ]:
figsize = (10,15)
rotulos = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
df = pd.read_csv(path, header=None, names=rotulos)
g_class = df.loc[df['class'] == 'g']
h_class = df.loc[df['class'] == 'h']
classes = df.pop('class').map(lambda x: 1 if (x == 'g') else -1)


In [ ]:
df.boxplot(figsize=figsize)
df.hist(figsize=figsize)

In [ ]:
g_class.boxplot(figsize=figsize)
g_class.hist(figsize=figsize)

In [ ]:
h_class.boxplot(figsize=figsize)
h_class.hist(figsize=figsize)


# SIGNAL


In [ ]:
def signal(_in):
  if _in >= 0:
      return 1
  return -1

# INICIADOR DE PESOS

In [ ]:
def iniciar_pesos(shape, zero=False):
  if zero:
    return np.zeros(shape)
  return (np.random.random_sample(shape) - .5)

def normatize(X):
    return 2 * ((X - X.min(axis=0))/(X.max(axis=0) - X.min(axis=0))) - 1

# Perceptron

In [ ]:
class Perceptron():

    def __init__(self, lr:float=0.001, num_iteration=100):
        self.lr = lr
        self.num_iteration = num_iteration

    def fit(self, X:NDFrame, y:NDFrame):
        X = np.concatenate((-np.ones((X.shape[0],1)), X), axis=1)
        self.W = iniciar_pesos((X.shape[1],1))
        it = 0
        while it <= self.num_iteration:
          for x,_y in zip(X,y):
            u_t = (self.W.T@x.reshape((x.shape[0],1)))[0]
            y_t = signal(u_t)
            it_err = (_y - y_t)
            self.W = self.W + (self.lr * it_err * x).reshape(x.shape[0],1)
          it += 1
    
    def predict(self, X:NDFrame):
      l = []
      X = np.concatenate((-np.ones((X.shape[0],1)),X), axis=1)
      for x in X:
        u_t = (self.W.T@x.reshape((x.shape[0],1)))[0]
        l.append(signal(u_t))
      return np.array(l)
    
    def accuracy(self,yPred,Y):
      points = 0
      for yP,y in zip(yPred,Y):
        if yP == y:
          points += 1
      return (points/len(Y)) * 100


# Adaline

In [ ]:
class Adaline():
  def __init__(self,lr:float = 0.0001, maxEpoch:float = 3000, pr:float = 0.1 ):
    self.lr = lr
    self.pr = pr
    self.maxEpoch = maxEpoch
    self.epoch = 0
    self.EQM_atual = 1
    self.EQM_anterior = 0
    self.EQMS = []

  def fit(self, X, y):
    
    def EQM(X, y, W):
      eqm = 0
      for x, _y in zip(X, y):
          u_t = (W.T@x.reshape(x.shape[0], 1))[0]
          e_t = _y - u_t
          eqm = eqm + ((e_t) ** 2)
      return eqm / ( 2 * X.shape[0] )

    X = np.concatenate((-np.ones((X.shape[0], 1)), X), axis=1)
    self.W = iniciar_pesos((X.shape[1],1))
    while (self.epoch <= self.maxEpoch and abs(self.EQM_atual - self.EQM_anterior) > self.pr):
      self.EQM_anterior = EQM(X,y,self.W)
      for x,_y in zip(X,y):
        u_t = (self.W.T@x.reshape(x.shape[0],1))[0]
        e_t = _y - u_t
        self.W = self.W + (self.lr * e_t * x).reshape(x.shape[0],1)
      self.epoch += 1
      self.EQM_atual = EQM(X,y,self.W)
      self.EQMS.append(abs(self.EQM_atual - self.EQM_anterior))
  
  def predict(self, X:NDFrame):
      l = []
      X = np.concatenate((-np.ones((X.shape[0],1)),X), axis=1)
      for x in X:
        u_t = (self.W.T@x.reshape((x.shape[0],1)))[0]
        l.append(signal(u_t))
      return np.array(l)

  def accuracy(self,yPred,Y):
      points = 0
      for yP,y in zip(yPred,Y):
        if yP == y:
          points += 1
      return (points/len(Y)) * 100

  


# MLP (Multi-Layer Perception)

In [ ]:

class MLP():
  def __init__(self, **kwargs):
    
    def default(prop, def_val):
      return kwargs[prop] if prop in kwargs else def_val

    self.lr = default('lr', 1e-5)
    self.maxEpoch = default('max_epoch', 2000)
    self.pr = default('pr', 0.1)
    self.hidden_layers = default('hidden_layers', 2)
    self.neuron_per_layer = default('neuron_per_layer', 2)
    self.neurons_in_out = default('neurons_in_out', 1)
    self.EQM_atual = 1
    self.epoch = 0
  
  def fit(self,X,y):
    # self.bias = np.array([iniciar_pesos((X.shape[0],1)) for i in range(self.hidden_layers)])
    X = np.concatenate((-np.ones((X.shape[0], 1)), normatize(X)), axis=1)
    self.W = [iniciar_pesos((X.shape[1], self.neuron_per_layer)) for _ in range(self.hidden_layers)]
    self.y = [-1] * (self.hidden_layers)
    self.i = [-1] * (self.hidden_layers)
    self.sigma = [-1] * self.hidden_layers
    while self.EQM_atual > self.pr and self.epoch < self.maxEpoch:
      for x,_y in zip(X, y):
        self.forward(np.matrix(x), self.sigmoid)
        self.backward(np.matrix(x), _y, self.sigmoid)
      self.EQM_atual = self.EQM(X, y)
      self.epoch = self.epoch + 1

  def forward(self, x, g):
    def net_input(W,x):
      # if W.shape == x.shape:
      #   return W.T@np.matrix(x)
      print(W.shape,x.shape)
      return W@x.T

    for j in range(len(self.W)):
        if j == 0:
          self.i[j] = net_input(self.W[j], x)
          self.y[j] = g(self.i[j])
        else:
          y_bias = np.matrix(self.y[j - 1])
          self.i[j] = net_input(self.W[j].T, y_bias)
          self.y[j] = g(self.i[j])

  def backward(self, x, d, g):
    j = len(self.W) - 1
    while j >= 0:
      if (j + 1) == len(self.W):
        self.sigma[j] = g(self.i[j],der=True) * (d - self.y[j]).T
        y_bias = self.y[j-1]
        self.W[j] = self.W[j] + self.lr * np.outer(self.sigma[j], y_bias)
      elif j == 0:
        w_b = np.matrix(self.W[j + 1]).T
        print((w_b.shape, self.sigma[j + 1].shape))
        self.sigma[j] = g(self.i[j],der=True) * (w_b@self.sigma[j + 1])
        self.W[j] = self.W[j] + self.lr * np.outer(self.sigma[j], x)
      else:
        w_b = np.matrix(self.W[j + 1]).T
        self.sigma[j] = g(self.i[j],der=True) * (w_b@self.sigma[j + 1]).T
        y_bias = self.y[j-1]
        self.W[j] = self.W[j] + self.lr * np.outer(self.sigma[j], y_bias)
      j = j - 1

  def sigmoid(self,x, der=False):
    if der:
      return x / ( 1 - x)
    return 1/(1 + np.exp(-x))

  def tanh(self, x, der=False):
    if der:
      return 0.5 * (1 - (x**2))
    return (1 - np.exp(-x)) / (1 + np.exp(-x))

  def EQM(self, X, y):
    eqm = 0
    for x, _y in zip(X, y):
      self.forward(x, self.sigmoid)
      eqi = 0
      for j in range(self.hidden_layers):
        eqi = eqi + (_y - self.y[self.hidden_layers -1][j])**2
      eqm = eqm + eqi
    return eqm / (2 * X.shape[1])

  def predict(self, X:NDFrame):
      for x in X:
        self.forward(x, self.sigmoid)

  def accuracy(self,yPred,Y):
      points = 0
      for yP,y in zip(yPred,Y):
        if yP == y:
          points += 1
      return (points/len(Y)) * 100

# RBF (Radial Basis Function)

In [ ]:
class RBF:

  def __init__(self, eta=0.1, q=5, sigma=1):
    self.eta = eta
    self.sigma = sigma
    self.W = None

  def fit(self, X, Y):
    self.centros = X[np.random.choice(len(X), q)]
    for x, _y in zip(X, Y):
      for c in self.centros:
        x_c = x - c
        i = np.argmin(x_c)
        G = -np.exp(x_c**2/2*(self.sigma**2))
        if (self.W != None):
          self.W = self.W + np.linalg.pinv(G@G.T)@G@self.y
        else:
          self.W = np.linalg.pinv(G@G.T)@G@self.y


  def predict(self, Y):
    return self.W@Y

  def accuracy(self,yPred,Y):
      points = 0
      for yP,y in zip(yPred,Y):
        if yP == y:
          points += 1
      return (points/len(Y)) * 100



# Treinamento

In [ ]:
RODADAS = 1#00

for i in range(RODADAS):
    print("RODADA " + str(i))
    Xtrain = df.loc[0:int(df.shape[0] * 0.7),:]
    Ytrain = classes[0:int(df.shape[0] * 0.7) + 1]
    
    Xtest  = df.loc[int(df.shape[0] * 0.7):,:]
    Ytest = classes[int(df.shape[0] * 0.7):]
    print('')
    print('Perceptron')
    print('')

    p = Perceptron()
    p.fit(Xtrain,Ytrain)
    ypred_p = p.predict(Xtest)

    print(p.accuracy(ypred_p,Ytest))
    print('=' * 50)
    
    print('Adaline')
    print('')

    a = Adaline()
    a.fit(Xtrain,Ytrain)
    ypred_a = a.predict(Xtest)
    
    print(a.accuracy(ypred_a,Ytest))

    print('=' * 50)
    print('')

    # print('MLP')
    # print('')

    # m = MLP()
    # m.fit(Xtrain,Ytrain)
    # ypred_m = m.predict(Xtest)
    
    # print(m.accuracy(ypred_m,Ytest))

    # print('=' * 50)
    # print('')

    